In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Input
import keras
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/google/google.csv",index_col = 0)

In [ ]:
df.head(5)

In [ ]:
def make_data(df,win):
    k = 0
    x = []
    y = []
    while (k+win<len(df)-1):
        xa = df[k:k+win]
        ya = df[k+win+1]
        x.append(xa)
        y.append(ya)
        k+=1
    
    x = np.array(x)
    y = np.array(y)

    return x,y

In [ ]:
def split_data(x,y):
    return train_test_split(x,y,test_size = 0.2,shuffle = False)

In [ ]:
req_df = df['Open']
scaler = MinMaxScaler(feature_range= (0,1))
req_df = scaler.fit_transform(np.array(req_df).reshape(-1,1))

In [ ]:
win = 10
x,y = make_data(req_df,win)
train_x,test_x,train_y,test_y = split_data(x,y)
train_x = train_x.reshape(train_x.shape[0],win,1)
test_x = test_x.reshape(test_x.shape[0],win,1)
print(train_x.shape)
print(test_y.shape)
print(train_y.shape)
print(test_x.shape)

In [ ]:
df['Open'].plot(figsize = (16,6))

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(win,1)))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(train_x,train_y,validation_data = (test_x,test_y),batch_size = 32, epochs = 100,verbose = 1)

In [ ]:
scaled_pred = model.predict(test_x)

In [ ]:
pred = scaler.inverse_transform(scaled_pred)

In [ ]:
org_data = np.array(df['Open'])
pred_x = np.arange(int(.8*len(df)),len(org_data)-2)
org_x = np.arange(len(req_df))
plt.plot(org_x,df['Open'])
plt.plot(pred_x,pred)
plt.show()

In [ ]:
mean_squared_error(scaled_pred,test_y)

In [ ]:
org_x.shape,df['Open'].shape